In [21]:
import numpy as np
import string 
import scipy.io
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

import numpy as np
import pandas as pd
import scipy

In [26]:
predicted = {}
target = {}

label = "test"

predicted['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/predicted_output_"+label+".npy")[0]
target['glove_10'] = np.load("../log_root_cart_sub1/subject_1/word2brain/glove_linear/10/L2_DropIn8H5_wt/target_output_"+label+".npy")

model = 'glove_10'
cosine_dists = distance.cdist(predicted[model],target[model],'cosine')
euc_dists =  distance.cdist(predicted[model],target[model],'euclidean')

predicted['contextual_01'] = np.load("../log_root_cart_sub1/word2brain/contextual_01_avg/intended/L2_DropIn8H5/predicted_output_"+label+".npy")
target['contextual_01'] = np.load("../log_root_cart_sub1/word2brain/contextual_01_avg/intended/L2_DropIn8H5/target_output_"+label+".npy")
model = 'contextual_01'
cosine_dists = distance.cdist(predicted[model],target[model],'cosine')
euc_dists =  distance.cdist(predicted[model],target[model],'euclidean')

TypeError: globals must be a real dict; try eval(expr, {}, mapping)

In [30]:
sorted_indexes = np.argsort(euc_dists,axis=1)

In [31]:
prec_at_corrects = []
ranks = []

for i in np.arange(len(predicted['glove_10'])):
    print(i)
    correct_at = np.where(sorted_indexes[i] == i)[0] + 1
    print("Reciprocal Rank",correct_at)
    prec_at_correct = 1.0/correct_at
    print("precision at ",correct_at,": ",prec_at_correct)
    prec_at_corrects.append(prec_at_correct)
    ranks.append(correct_at)
    
print("MRR: ",np.mean(prec_at_corrects)," ",np.mean(ranks))

0
Reciprocal Rank [343]
precision at  [343] :  [0.00291545]
1
Reciprocal Rank [334]
precision at  [334] :  [0.00299401]
2
Reciprocal Rank [310]
precision at  [310] :  [0.00322581]
3
Reciprocal Rank [326]
precision at  [326] :  [0.00306748]
4
Reciprocal Rank [347]
precision at  [347] :  [0.00288184]
5
Reciprocal Rank [330]
precision at  [330] :  [0.0030303]
6
Reciprocal Rank [318]
precision at  [318] :  [0.00314465]
7
Reciprocal Rank [315]
precision at  [315] :  [0.0031746]
8
Reciprocal Rank [334]
precision at  [334] :  [0.00299401]
9
Reciprocal Rank [330]
precision at  [330] :  [0.0030303]
10
Reciprocal Rank [336]
precision at  [336] :  [0.00297619]
11
Reciprocal Rank [320]
precision at  [320] :  [0.003125]
12
Reciprocal Rank [337]
precision at  [337] :  [0.00296736]
13
Reciprocal Rank [347]
precision at  [347] :  [0.00288184]
14
Reciprocal Rank [342]
precision at  [342] :  [0.00292398]
15
Reciprocal Rank [340]
precision at  [340] :  [0.00294118]
16
Reciprocal Rank [329]
precision at  

precision at  [104] :  [0.00961538]
222
Reciprocal Rank [145]
precision at  [145] :  [0.00689655]
223
Reciprocal Rank [123]
precision at  [123] :  [0.00813008]
224
Reciprocal Rank [137]
precision at  [137] :  [0.00729927]
225
Reciprocal Rank [123]
precision at  [123] :  [0.00813008]
226
Reciprocal Rank [136]
precision at  [136] :  [0.00735294]
227
Reciprocal Rank [124]
precision at  [124] :  [0.00806452]
228
Reciprocal Rank [172]
precision at  [172] :  [0.00581395]
229
Reciprocal Rank [152]
precision at  [152] :  [0.00657895]
230
Reciprocal Rank [160]
precision at  [160] :  [0.00625]
231
Reciprocal Rank [165]
precision at  [165] :  [0.00606061]
232
Reciprocal Rank [142]
precision at  [142] :  [0.00704225]
233
Reciprocal Rank [88]
precision at  [88] :  [0.01136364]
234
Reciprocal Rank [93]
precision at  [93] :  [0.01075269]
235
Reciprocal Rank [154]
precision at  [154] :  [0.00649351]
236
Reciprocal Rank [120]
precision at  [120] :  [0.00833333]
237
Reciprocal Rank [103]
precision at  [

In [32]:
true_mu = 361 / 2.0

onesample_results = scipy.stats.ttest_1samp(ranks, true_mu)
print("ranks: ", onesample_results)

ranks:  Ttest_1sampResult(statistic=array([-0.43422506]), pvalue=array([0.66438515]))


In [35]:
def eval(dists,e_dists,model):
    nn_index = np.argmin(dists,axis=1)
    accuracy_on_test = np.mean(nn_index == np.argmax(np.eye(len(target[model])),axis=1))


    b_acc = []
    e_b_acc = []
    for i,j in itertools.combinations(np.arange(len(target[model])), 2):
        right_match = dists[i,i] + dists[j,j]
        wrong_match = dists[i,j] + dists[j,i]
        b_acc.append(right_match < wrong_match)

        e_right_match = e_dists[i,i] + e_dists[j,j]
        e_wrong_match = e_dists[i,j] + e_dists[j,i]
        e_b_acc.append(e_right_match < e_wrong_match)

    #print("binary accuracy: ", np.mean(b_acc)," ", np.mean(e_b_acc))
    return np.mean(b_acc),np.mean(e_b_acc),b_acc,e_b_acc

In [60]:
mean_b_acc, mean_e_b_acc,b_acc, e_b_acc = eval(cosine_dists,euc_dists,model)
number_of_trues = len(np.where(np.asarray(b_acc) == True)[0]) / len(b_acc)
number_of_falses = len(np.where(np.asarray(b_acc) == False)[0])/ len(b_acc)

In [66]:
print(scipy.stats.chisquare([number_of_trues,number_of_falses],[0.5,0.5]))

Power_divergenceResult(statistic=0.219124771919085, pvalue=0.639707608910669)


In [83]:
true_mu = .50
print(np.mean(b_acc))
onesample_results = scipy.stats.ttest_1samp([np.mean(b_acc),np.mean(e_b_acc)], true_mu)
print("binary accuracy: ", onesample_results)

0.7340538249629158
binary accuracy:  Ttest_1sampResult(statistic=102.43143812709157, pvalue=0.006214884414269133)


In [75]:
number_of_falses

0.2659461750370841